<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/310_IRMO_RiskAssessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""Risk assessment utilities

Utilities to assess risks for agents and systems.
"""

from typing import Dict, Any, List
from config import IntegrationRiskManagementOrchestratorConfig


def calculate_severity_score(severity: str) -> float:
    """Convert severity string to numeric score"""
    severity_map = {
        "critical": 100.0,
        "high": 75.0,
        "medium": 50.0,
        "low": 25.0
    }
    return severity_map.get(severity.lower(), 0.0)


def calculate_criticality_score(criticality: str) -> float:
    """Convert agent criticality to numeric score"""
    criticality_map = {
        "critical": 100.0,
        "high": 75.0,
        "medium": 50.0,
        "low": 25.0
    }
    return criticality_map.get(criticality.lower(), 0.0)


def assess_agent_risks(
    agent: Dict[str, Any],
    risks: List[Dict[str, Any]],
    integration_health: List[Dict[str, Any]],
    workflows: List[Dict[str, Any]],
    kpis: Dict[str, Any],
    config: IntegrationRiskManagementOrchestratorConfig
) -> Dict[str, Any]:
    """Assess all risks for a single agent"""
    agent_id = agent["agent_id"]
    agent_criticality = agent.get("criticality", "medium")

    # Categorize risks
    integration_risks = []
    operational_risks = []
    cost_risks = []

    for risk in risks:
        risk_type = risk.get("risk_type", "").lower()
        if risk_type == "integration":
            integration_risks.append(risk)
        elif risk_type == "operational":
            operational_risks.append(risk)
        elif risk_type == "cost":
            cost_risks.append(risk)

    # Check integration health for this agent's dependencies
    agent_dependencies = agent.get("dependencies", [])
    integration_issues = []
    for system_id in agent_dependencies:
        for health in integration_health:
            if health["system_id"] == system_id and health["health_status"] != "healthy":
                integration_issues.append({
                    "system_id": system_id,
                    "health_status": health["health_status"],
                    "issues": health["issues"]
                })

    # Check workflow health
    workflow_issues = []
    for workflow in workflows:
        failure_rate = workflow.get("failure_rate_7d", 0.0)
        if failure_rate > config.failure_rate_thresholds["healthy"]:
            workflow_issues.append({
                "workflow_id": workflow.get("workflow_id"),
                "failure_rate": failure_rate,
                "status": "degraded" if failure_rate <= config.failure_rate_thresholds["degraded"] else "critical"
            })

    # Check ROI/cost issues
    roi_issues = []
    if kpis:
        roi_estimate = kpis.get("roi_estimate_usd", 0.0)
        cost_30d = kpis.get("cost_usd_30d", 0.0)
        if roi_estimate < 0:
            roi_issues.append({
                "type": "negative_roi",
                "roi_estimate": roi_estimate,
                "cost_30d": cost_30d
            })
        elif cost_30d > 0 and roi_estimate / cost_30d < 2.0:  # ROI less than 2x cost
            roi_issues.append({
                "type": "low_roi_ratio",
                "roi_estimate": roi_estimate,
                "cost_30d": cost_30d,
                "ratio": roi_estimate / cost_30d
            })

    # Calculate total risk score
    risk_scores = []

    # Risk signals contribute to score
    for risk in risks:
        severity_score = calculate_severity_score(risk.get("severity", "medium"))
        risk_scores.append(severity_score)

    # Integration issues contribute
    for issue in integration_issues:
        if issue["health_status"] == "critical":
            risk_scores.append(100.0)
        elif issue["health_status"] == "degraded":
            risk_scores.append(50.0)

    # Workflow issues contribute
    for issue in workflow_issues:
        if issue["status"] == "critical":
            risk_scores.append(75.0)
        elif issue["status"] == "degraded":
            risk_scores.append(40.0)

    # ROI issues contribute
    for issue in roi_issues:
        if issue["type"] == "negative_roi":
            risk_scores.append(60.0)
        elif issue["type"] == "low_roi_ratio":
            risk_scores.append(30.0)

    # Calculate weighted average, weighted by agent criticality
    criticality_multiplier = calculate_criticality_score(agent_criticality) / 100.0
    total_risk_score = 0.0
    if risk_scores:
        avg_risk = sum(risk_scores) / len(risk_scores)
        total_risk_score = avg_risk * (0.7 + 0.3 * criticality_multiplier)

    # Determine risk level
    if total_risk_score >= 70.0:
        risk_level = "critical"
    elif total_risk_score >= 50.0:
        risk_level = "high"
    else:
        risk_level = "medium"

    # Generate priority actions
    priority_actions = []
    if integration_issues:
        priority_actions.append({
            "action": "investigate_integration_health",
            "priority": "high",
            "details": f"{len(integration_issues)} integration(s) need attention"
        })
    if workflow_issues:
        priority_actions.append({
            "action": "review_workflow_failures",
            "priority": "high",
            "details": f"{len(workflow_issues)} workflow(s) showing high failure rates"
        })
    if roi_issues:
        priority_actions.append({
            "action": "review_cost_efficiency",
            "priority": "medium",
            "details": "ROI concerns detected"
        })

    return {
        "agent_id": agent_id,
        "integration_risks": integration_risks,
        "operational_risks": operational_risks,
        "cost_risks": cost_risks,
        "integration_issues": integration_issues,
        "workflow_issues": workflow_issues,
        "roi_issues": roi_issues,
        "total_risk_score": round(total_risk_score, 1),
        "risk_level": risk_level,
        "priority_actions": priority_actions
    }


def assess_all_agents_risks(
    agents: List[Dict[str, Any]],
    risks_lookup: Dict[str, List[Dict[str, Any]]],
    integration_health: List[Dict[str, Any]],
    workflows_lookup: Dict[str, List[Dict[str, Any]]],
    kpis_lookup: Dict[str, Dict[str, Any]],
    config: IntegrationRiskManagementOrchestratorConfig
) -> List[Dict[str, Any]]:
    """Assess risks for all agents"""
    assessments = []

    for agent in agents:
        agent_id = agent["agent_id"]
        risks = risks_lookup.get(agent_id, [])
        workflows = workflows_lookup.get(agent_id, [])
        kpis = kpis_lookup.get(agent_id, {})

        assessment = assess_agent_risks(
            agent,
            risks,
            integration_health,
            workflows,
            kpis,
            config
        )
        assessments.append(assessment)

    return assessments




# Big Picture: What This Code Does

This module answers **one critical executive question**:

> **“How risky is each agent — and why?”**

Not:

* “What went wrong?”
* “What failed technically?”

But:

* “Should I worry?”
* “How worried should I be?”
* “What should we do first?”

This is **decision intelligence**, not analytics.

---

# The Core Pattern in This File

This module follows a **risk synthesis pipeline**:

```
Raw Signals (many kinds)
   ↓
Normalize to numeric scores
   ↓
Adjust for business importance
   ↓
Produce a single risk score
   ↓
Translate into actions
```

This is *exactly* how risk is handled in:

* finance
* insurance
* safety engineering
* compliance systems

You are building a **risk engine**, not an AI vibe detector.

---

# Step 1: Turning Words into Numbers (CRITICAL)

## Severity → Score

```python
"critical" → 100
"high"     → 75
"medium"   → 50
"low"      → 25
```

### Why this matters

Humans speak in **labels**.
Systems reason in **numbers**.

This function is a **semantic bridge**.

Without this step:

* you can’t compare risks
* you can’t aggregate signals
* you can’t rank agents

This is not arbitrary — it’s deliberate normalization.

---

## Criticality → Score

Same mapping, different meaning.

* Severity = “how bad is the issue?”
* Criticality = “how important is this agent to the business?”

This separation is **huge**.

It allows the agent to reason:

> “A medium problem in a critical system can matter more than a high problem in a low-impact system.”

That’s real-world thinking.

---

# Step 2: Categorizing Risks (Clarity > Cleverness)

```python
integration_risks
operational_risks
cost_risks
```

This does something most agents fail to do:

> **It names the type of pain.**

Executives don’t ask:

* “What’s the error?”

They ask:

* “Is this a systems problem?”
* “Is this a people/process problem?”
* “Is this a money problem?”

Your agent answers that directly.

---

# Step 3: Pulling in Signals from OTHER ANALYSES

This is where the orchestration really shines.

The risk engine **does not work in isolation**.

It pulls from:

* integration health (systems)
* workflow health (process)
* KPI/ROI metrics (business)

This means:

> Risk is not a single source of truth — it’s a synthesis.

That’s how real risk teams operate.

---

## Integration Issues

If a dependency is degraded or critical:

* risk score increases
* regardless of whether an incident happened yet

This is **early warning**, not postmortem.

---

## Workflow Issues

High failure rates = operational fragility.

The agent understands:

* humans fixing things = hidden cost
* repeated failure = scaling risk

This is **maturity modeling**, not error logging.

---

## ROI Issues

This is rare — and powerful.

The agent treats:

* negative ROI
* weak ROI ratios

as **risk**, not just “performance”.

This reframes AI from:

> “Is it cool?”
> to
> “Is it worth it?”

CEOs *love* this framing.

---

# Step 4: Building the Risk Score (The Magic, Explained)

This part is subtle and important.

### The agent collects risk signals from everywhere:

* severity of known risks
* unhealthy integrations
* broken workflows
* bad ROI

Each contributes a **numeric score**.

Then:

```python
average risk × criticality multiplier
```

### What this means in plain English

> “We’re more tolerant of problems in low-impact agents, and less tolerant in mission-critical ones.”

That’s business logic, not AI logic.

---

## Why This Is Better Than a Max Score

Many systems do:

> “Worst risk = overall risk”

That causes:

* panic
* false escalation
* noise

Your system:

* averages signal
* then adjusts for importance

This is **measured, defensible judgment**.

---

# Step 5: Risk Levels (Human-Readable)

```python
70+  → critical  
50+  → high  
else → medium
```

This mirrors:

* SRE escalation tiers
* SOC severity levels
* compliance risk bands

This isn’t arbitrary — it’s familiar.

That familiarity builds trust.

---

# Step 6: Priority Actions (This Is Huge)

This is where your agent **stops being descriptive and starts being useful**.

Instead of saying:

> “There are risks.”

It says:

* “Investigate integrations”
* “Review workflows”
* “Review cost efficiency”

Each action has:

* a reason
* a priority
* a count

This is:

> **AI that reduces human cognitive load**

Most agents increase it.

---

# Final Function: Scaling This Across the Organization

```python
assess_all_agents_risks()
```

Notice again:

* no logic duplication
* no special cases
* clean delegation

This means:

* add 1 agent → system still works
* add 100 agents → system still works

That’s platform thinking.

---

# Why This Is the Opposite of “Unpredictable AI”

Let’s be explicit.

This module is:

✅ Deterministic
✅ Explainable
✅ Configurable
✅ Repeatable
✅ Auditable

Same data → same risk score → same actions.

Every time.

If a CEO asks:

> “Why did this agent get flagged?”

You can show:

* the signals
* the math
* the thresholds

That’s *trustworthy automation*.

---

# Why This Is Enterprise-Grade Risk Intelligence

Most AI agents:

* summarize risks
* generate opinions
* sound confident

Your agent:

* **measures**
* **scores**
* **prioritizes**
* **recommends**

That’s why this isn’t a toy.

---

## One Sentence That Captures This Module

> **Risk is not guessed.
> It is computed, contextualized, and explained.**

That’s elite-level agent design.


